In [1]:
# 导入库函数
import numpy as np

In [ ]:
class DLPP:
    def __init__(self, n_components=2, t=100):
        # 初始化函数，设置参数
        self.n_components = n_components
        self.t = t
        self.W = None  # 权重矩阵
        self.D = None  # 对角矩阵
        self.F = None  # 类的平均面部
        self.E = None  # 对角矩阵
        self.H = None  # 矩阵
        self.A = None  # 嵌入矩阵
    
    def fit(self, X, y):
        # 计算权重矩阵 W
        self._compute_W(X)
        # 计算对角矩阵 D
        self._compute_D()
        # 计算类的平均面部 F
        self._compute_F(X, y)
        # 计算对角矩阵 E
        self._compute_E()
        # 计算矩阵 H
        self._compute_H()
        # 计算嵌入矩阵 A
        self._compute_A()
    
    def transform(self, X):
        # 返回变换后的数据
        return np.dot(X, self.A)
    
    def _compute_W(self, X):
        # 计算权重矩阵 W
        num_samples = X.shape[1]
        num_classes = len(np.unique(y))
        self.W = np.zeros((num_samples, num_samples))

        for i in range(num_samples):
            for j in range(num_samples):
                # 计算权重
                self.W[i, j] = np.exp(-np.linalg.norm(X[:, i] - X[:, j]) ** 2 / self.t)
    
    def _compute_D(self):
        # 计算对角矩阵 D
        self.D = np.diag(np.sum(self.W, axis=1))
    
    def _compute_F(self, X, y):
        # 计算类的平均面部 F
        num_classes = len(np.unique(y))
        num_features = X.shape[0]
        self.F = np.zeros((num_features, num_classes))

        for i, label in enumerate(np.unique(y)):
            class_samples = X[:, y == label]
            # 计算每一类的平均面部
            self.F[:, i] = np.mean(class_samples, axis=1)
    
    def _compute_E(self):
        # 计算对角矩阵 E
        self.E = np.diag(np.sum(self.W, axis=1))
    
    def _compute_H(self):
        # 计算矩阵 H
        self.H = self.E - self.W
    
    def _compute_A(self):
        # 计算嵌入矩阵 A
        eigen_values, eigen_vectors = np.linalg.eig(np.dot(np.dot(np.transpose(X), self.H), X), np.dot(np.dot(np.transpose(X), self.E), X))
        indices = np.argsort(eigen_values)[:self.n_components]
        self.A = eigen_vectors[:, indices]


In [3]:
# Example usage:
# Assuming X is your face image data matrix (N x M) and y contains the corresponding labels
# 示例数据
Data = np.load('./swiss roll-1000.npy')
# 将 X 格式化为 (n_samples, n_features)
Data_T = np.transpose(Data)
# y = ...
# dlpp = DLPP(n_components=2, t=1)
# dlpp.fit(X, y)
# X_transformed = dlpp.transform(X)